# Dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Raw Data Merge

In [19]:
#import data
filepath1 = 'Resources/Raw/2019loans.csv'
filepath2 = 'Resources/Raw/2020Q1loans.csv'
df1 = pd.DataFrame(pd.read_csv(filepath1))
df2 = pd.DataFrame(pd.read_csv(filepath2))

In [20]:
combined_df = pd.concat([df1, df2])
combined_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk


# EDA 

In [41]:
combined_df["home_ownership"].value_counts(normalize=True)

MORTGAGE    0.462090
RENT        0.415531
OWN         0.117995
ANY         0.004383
Name: home_ownership, dtype: float64

In [37]:
combined_df["int_rate"].describe() # interest rate

count    16882.000000
mean         0.142983
std          0.054268
min          0.060000
25%          0.103300
50%          0.130800
75%          0.177400
max          0.289700
Name: int_rate, dtype: float64

In [39]:
combined_df["revol_bal"].describe() # revolving balance

count     16882.000000
mean      17718.171899
std       22852.300889
min           0.000000
25%        6380.250000
50%       12197.500000
75%       21597.750000
max      517103.000000
Name: revol_bal, dtype: float64

In [38]:
combined_df["dti"].describe() # debt to income ratio

count    16882.000000
mean        22.852588
std         24.265121
min          0.000000
25%         14.470000
50%         20.670000
75%         27.530000
max        999.000000
Name: dti, dtype: float64

In [77]:
totals_raw_df = combined_df[['hardship_flag', 'target']].value_counts().to_frame()
totals_raw_df

0
hardship_flag target         
N             low_risk   8326
              high_risk  8132
Y             high_risk   309
              low_risk    115

In [69]:
print(f"Total Entries: {combined_df.shape[0]}")
totals_df = combined_df[['hardship_flag', 'target']].value_counts(normalize=True).to_frame()
totals_df

Total Entries: 16882


0
hardship_flag target             
N             low_risk   0.493188
              high_risk  0.481696
Y             high_risk  0.018304
              low_risk   0.006812

In [73]:
totals_list = combined_df[['hardship_flag', 'target']].value_counts(normalize=True).to_list()
totals_list = [round(i*100,3) for i in totals_list]
totals_list

[49.319, 48.17, 1.83, 0.681]

In [84]:
print(f"accepted {totals_list[0] + totals_list[3]}%, rejected {100 - (totals_list[1] + totals_list[2])}%")
print(f"of rejected, {round((totals_raw_df[0][2]/totals_raw_df[0][1])*100,3)}%")

accepted 50.0%, rejected 50.0%
of rejected, 3.8%


# Preprocessing

In [3]:
#check num of null values
df.isnull().sum()

class                       0
cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               51538
stem-surface            38124
stem-color                  0
veil-type               57892
veil-color              53656
has-ring                    0
ring-type                2471
spore-print-color       54715
habitat                     0
season                      0
dtype: int64

In [4]:
#drop features with more than 75% null values
df_smaller = df.drop(columns={'stem-root', 'stem-surface','veil-type', 'veil-color', 'spore-print-color', 'gill-spacing'})
df_smaller.head()

#dropped columns that had large number of nan
#nan could be a useful classifier, will consider re-running with the nans left in

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
0,p,15.26,x,g,o,f,e,w,16.95,17.09,w,t,g,d,w
1,p,16.60,x,g,o,f,e,w,17.99,18.19,w,t,g,d,u
2,p,14.07,x,g,o,f,e,w,17.80,17.74,w,t,g,d,w
3,p,14.17,f,h,e,f,e,w,15.77,15.98,w,t,p,d,w
4,p,14.64,x,h,o,f,e,w,16.53,17.20,w,t,p,d,w


In [5]:
#dropping rows with nans
df_smaller.dropna(inplace=True)

In [6]:
#one-hot encoding
df_dummies = pd.get_dummies(df_smaller)
df_dummies

,cap-diameter,stem-height,stem-width,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,...,habitat_g,habitat_h,habitat_l,habitat_m,habitat_u,habitat_w,season_a,season_s,season_u,season_w
0,15.26,16.95,17.09,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,16.60,17.99,18.19,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,14.07,17.80,17.74,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,14.17,15.77,15.98,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,14.64,16.53,17.20,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,1.18,3.93,6.22,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
61065,1.27,3.18,5.43,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
61066,1.27,3.86,6.37,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
61067,1.24,3.56,5.44,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
#separate X and y data
X = df_dummies.drop(columns = {'class_e','class_p'})
y = df_dummies['class_e']
first_row = X.head(1)
first_row

,cap-diameter,stem-height,stem-width,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_o,cap-shape_p,cap-shape_s,cap-shape_x,...,habitat_g,habitat_h,habitat_l,habitat_m,habitat_u,habitat_w,season_a,season_s,season_u,season_w
0,15.26,16.95,17.09,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# splitting dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# scaling the data
# using standard scaler because assuming the mushroom data is normal
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Export

In [10]:
# traindata export
train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns.to_list())
train_df = pd.concat([train_df, y_train], axis=1, join='inner')
train_df.to_csv('../Resources/Cleaned/train.csv', index=False)

# test data export
test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns.to_list())
test_df = pd.concat([test_df, y_test], axis=1, join='inner')
test_df.to_csv('../Resources/Cleaned/test.csv', index=False)